In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Baseline_MUSS_corrupt/muss
!pip install -e . 
# !pip install -U spacy ==2.0.0
!python -m spacy download en_core_web_md

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mounted at /content/drive
/content/drive/MyDrive/Baseline_MUSS_corrupt/muss
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Baseline_MUSS_corrupt/muss
  Cloning https://github.com/feralvam/easse.git to /tmp/pip-install-r6473xqi/easse_0a615ac9a201441db89f3d9a8613ff57
  Running command git clone -q https://github.com/feralvam/easse.git /tmp/pip-install-r6473xqi/easse_0a615ac9a201441db89f3d9a8613ff57
  Cloning https://github.com/kpu/kenlm.git to /tmp/pip-install-r6473xqi/kenlm_77fe65afd9d34f5999aab1e0cc14e437
  Running command git clone -q https://github.com/kpu/kenlm.git /tmp/pip-install-r6473xqi/kenlm_77fe65afd9d34f5999aab1e0cc14e437
     |████████████████████████████████| 181 kB 4.2 MB/s 
     |████████████████████████████████| 10.4 MB 21.1 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
     |████████████████████████████████| 1.7 MB 55.5 MB/s 
     |████████████████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50.8 MB 1.2 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.3.1-py3-none-any.whl size=50916639 sha256=260659165f5c20971b2e9af79fce53bdf4f4d3e3432f3d68da0463bc7c0b5186
  Stored in directory: /root/.cache/pip/wheels/43/1d/c1/a0af68d0648debf57f875e9dda56bbac35cfc27bfa187ffc46
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
Tue Sep  6 15:23:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
| 

In [2]:

from muss.mining.training import get_bart_kwargs, get_score_rows
from muss.resources.prepare import prepare_wikilarge_detokenized, prepare_asset
from muss.resources.datasets import create_smaller_dataset

from muss.fairseq.main import fairseq_train_and_evaluate_with_parametrization
from muss.mining.training import get_score_rows #get_bart_kwargs

#########################
import re
import shutil
import os 

from cachier import cachier
from easse.cli import evaluate_system_output
from easse.utils.constants import TEST_SETS_PATHS
import torch
from tqdm import tqdm

from muss.resources.paths import get_data_filepath, MODELS_DIR, get_dataset_dir
from muss.utils.helpers import add_dicts, args_str_to_dict
from muss.utils.resources import download_and_extract
from muss.preprocessors import GPT2BPEPreprocessor
from muss.preprocessing import apply_line_function_to_file
from muss.fairseq.main import get_language_from_dataset
from muss.text import truncate

from muss.simplify import simplify_sentences
from muss.utils.helpers import *

from muss.simplifiers import get_fairseq_simplifier, get_preprocessed_simplifier
from muss.preprocessors import get_preprocessors, get_preprocessor_by_name

In [5]:

# This dataset should exist in resources/datasets/ and contain the following files:
# train.complex, train.simple, valid.complex, valid.simple, test.complex, test.simple
def get_evaluate_kwargs(language, phase='valid'):
    return {
        ('en', 'valid'): {'test_set': 'asset_valid'},
        ('en', 'test'): {'test_set': 'asset_test'},
        ('fr', 'valid'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('alector', 'valid', 'complex'),
            'refs_sents_paths': [get_data_filepath('alector', 'valid', 'simple')],
        },
        ('fr', 'test'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('alector', 'test', 'complex'),
            'refs_sents_paths': [get_data_filepath('alector', 'test', 'simple')],
        },
        ('es', 'valid'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('simplext_corpus', 'valid', 'complex'),
            'refs_sents_paths': [get_data_filepath('simplext_corpus', 'valid', 'simple')],
        },
        ('es', 'test'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('simplext_corpus', 'test', 'complex'),
            'refs_sents_paths': [get_data_filepath('simplext_corpus', 'test', 'simple')],
        },
    }[(language, phase)]

evaluate_kwargs=get_evaluate_kwargs('en')


def get_predict_files(language):
    return {
        'en': [get_data_filepath('asset', 'valid', 'complex'), get_data_filepath('asset', 'test', 'complex')],
        'fr': [get_data_filepath('alector', 'valid', 'complex'), get_data_filepath('alector', 'test', 'complex')],
        'es': [
            get_data_filepath('simplext_corpus', 'valid', 'complex'),
            get_data_filepath('simplext_corpus', 'test', 'complex'),
        ],
    }[language]

def get_access_preprocessors_kwargs(language, use_short_name=False):
    return {
        'LengthRatioPreprocessor': {'target_ratio': 0.8, 'use_short_name': use_short_name},
        'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.8, 'use_short_name': use_short_name},
        'WordRankRatioPreprocessor': {'target_ratio': 0.8, 'language': language, 'use_short_name': use_short_name},
        'DependencyTreeDepthRatioPreprocessor': {
            'target_ratio': 0.8,
            'language': language,
            'use_short_name': use_short_name,
        },
    }

def get_bart_kwargs(dataset, language, use_access, use_short_name=False, bart_model='bart.large'):
    assert language == 'en'

    bart_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/models/bart_mined/model.pt'
    arch = {
        'bart.base': 'bart_base',
        'bart.large': 'bart_large',
        'bart.large.cnn': 'bart_large',
    }[bart_model]
    kwargs = {
        'dataset': dataset,
        'metrics_coefs': [0, 1, 0],
        'parametrization_budget': 128,
        'predict_files': get_predict_files(language), # used in fairseq_evaluate_and_save
        'preprocessors_kwargs': {
            'GPT2BPEPreprocessor': {},
        },
        'preprocess_kwargs': {'dict_path': GPT2BPEPreprocessor().dict_path},
        'train_kwargs': {
            'ngpus': 1,
            'arch': arch,
            'restore_file': bart_path,
            'max_tokens': 4096,
            'lr': 8e-05,
            'warmup_updates': 500, #500
            'truncate_source': True,
            'layernorm_embedding': True,
            'share_all_embeddings': True,
            'share_decoder_input_output_embed': True,
            'reset_optimizer': True,
            'reset_dataloader': True,
            'reset_meters': True,
            'required_batch_size_multiple': 1,
            'criterion': 'label_smoothed_cross_entropy',
            'label_smoothing': 0.1,
            'dropout': 0.1,
            'attention_dropout': 0.1,
            'weight_decay': 0.01,
            'optimizer': 'adam',
            'adam_betas': '(0.9, 0.999)',
            'adam_eps': 1e-08,
            'clip_norm': 0.1,
            'lr_scheduler': 'polynomial_decay',
            'max_update': 10000, # 20000
            'skip_invalid_size_inputs_valid_test': True,
            'find_unused_parameters': True,
        },
        'evaluate_kwargs': get_evaluate_kwargs(language),
    }
    if use_access:
        kwargs['preprocessors_kwargs'] = add_dicts(
            get_access_preprocessors_kwargs(language, use_short_name=use_short_name), kwargs['preprocessors_kwargs']
        )
    return kwargs

def generate_output(chosen_id_list,data_dir_list,data_list,model_dir_dict,sample=False,add_to_dir_name=None,muss_output=False,NE_output=False,CERF_output=False,NE_CERF_output=False,hypothesis_num=None,beam=None,sampling=False):

  # store created dir
  created_dir = []
  output_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/'
  for index in chosen_id_list:
    
    # get model unique local id
    exp_dir = model_dir_dict[index]['exp_dir']
    print('| model_name is',model_dir_dict[index]['model_name'])
    print('| exp_dir is',exp_dir)

    model_name = 'model_'+ str(index) + '_'+ exp_dir.split('/')[-2]

    os.makedirs(output_dir+model_name, exist_ok=True)
    print('| output_dir',output_dir)
    language = 'en'

    # get recommended_preprocessors_kwargs
    if 'recommended_preprocessors_kwargs' in model_dir_dict[index].keys() :
      preprocessors_kwargs = model_dir_dict[index]['recommended_preprocessors_kwargs']
      print('| use provided kwargs')
    else:
      # use muss_mined_wiki preprocessors
      preprocessors_kwargs = {
            'LengthRatioPreprocessor': {'target_ratio': 0.9, 'use_short_name': False},
            'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.65, 'use_short_name': False},
            'WordRankRatioPreprocessor': {'target_ratio': 0.75, 'language': language, 'use_short_name': False},
            'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.4, 'language': language, 'use_short_name': False},
        }
      preprocessors_kwargs['GPT2BPEPreprocessor'] = {}

    print('| use preprocessors_kwargs',preprocessors_kwargs)
    preprocessors = get_preprocessors(preprocessors_kwargs)

    if hypothesis_num == None:
      hypothesis_num = 1
    if beam == None:
      beam=5

    generate_kwargs = {'hypothesis_num':hypothesis_num,
                       'beam':beam,
                       'sampling':sampling}

    simplifier = get_fairseq_simplifier(exp_dir, **generate_kwargs)
    simplifier = get_preprocessed_simplifier(simplifier, preprocessors=preprocessors)

    #########################################
    # simplify sample sentences 

    ############################## I need to change the directory 
    if sample:
      if muss_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_no_token.en'
      elif NE_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_NE_token.en'
      elif CERF_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_ABCD_token.en'
      elif NE_CERF_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_NE_ABCD_token.en'

      pred_path = simplifier(complex_file_dir)

      for i in range(len(read_lines(complex_file_dir))):
        print('original:\n',read_lines(complex_file_dir)[i])
        print('simplified:\n',read_lines(pred_path)[i])
        print('----------------------------------------------------')

      return read_lines(complex_file_dir),read_lines(pred_path)
    ###########################################

    # create dir for output
    subfolders = os.listdir(output_dir + model_name)
    num_subfolder = len(subfolders)
    new_folder_dir = output_dir + model_name + '/' +str(num_subfolder).zfill(2) +'/'
    os.mkdir(new_folder_dir)
    created_dir.append(new_folder_dir)

     # simplify asset dataset
    for index, file_dir in enumerate(data_dir_list):
      
      info_file_name = new_folder_dir + 'info.txt'
      with open(info_file_name, 'w') as f:
        f.write("generate_kwargs %s\n" % generate_kwargs)
      f.close()

      filename = new_folder_dir + data_list[index]
      print('filename;', filename)
      pred_path = simplifier(file_dir)

      with open(filename, 'w') as f:
          for item in read_lines(pred_path):
              f.write("%s\n" % item)

  print('created_dir',created_dir)
  return created_dir

def reconstruct_simplified_data(complex_sentence, simple_sentence):
    
    def list_duplicates_of(seq,item):
      start_at = -1
      locs = []
      while True:
          try:
              loc = seq.index(item,start_at+1)
          except ValueError:
              break
          else:
              locs.append(loc)
              start_at = loc
      return locs
    
    control_tokens = ["<NEXT_NE> ","<NEXT_DIFFICULT_WORD>","<SENT_START> "]

    temp_NE = NE_extraction(complex_sentence)[0]
    # for key, value in temp_NE.items():
    if temp_NE['NE_simple'] != '' or temp_NE['NE_difficult'] !=[]:
  #################
      indice_of_start = list_duplicates_of(complex_sentence,control_tokens[2])
      sentence = complex_sentence[0:indice_of_start[0]] + control_tokens[2] + simple_sentence
  #################
      # Change here to eliminate difficult words 
      # indice_of_start_diff = list_duplicates_of(complex_sentence,control_tokens[1])
      # sentence = complex_sentence[0:indice_of_start_diff[0]] + control_tokens[2] + simple_sentence
  #################    
      return sentence
    else:
      print('-'*50)
      print('occur sentence with no prefix')
      print(complex_sentence)
      print('-'*50)
      pass

In [7]:
pred_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/'+'model_0_local_1660919962534'+'/'+'45'+'/'+'test.diff.complex.pred'
asset_test_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.complex'
asset_valid_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/valid.complex'
# print(len(read_lines(pred_path)))
ast.literal_eval(os.popen("easse evaluate -t asset_valid -m 'bleu,sari,fkgl' -q < %s" %pred_path).read())


{'bleu': 44.89,
 'sari': 36.613,
 'fkgl': 7.598,
 'quality_estimation': {'Compression ratio': 1.417,
  'Sentence splits': 1.736,
  'Levenshtein similarity': 0.682,
  'Exact copies': 0.002,
  'Additions proportion': 0.439,
  'Deletions proportion': 0.16,
  'Lexical complexity score': 8.04}}

In [ ]:
# %cd /content/drive/MyDrive/Baseline_MUSS_corrupt/muss
!easse evaluate -t asset_valid -m 'bleu,sari,fkgl' -q < /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/model_0_local_1660919962534/44/test.diff.complex.pred

Traceback (most recent call last):
  File "/usr/local/bin/easse", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/easse/cli.py", line 124, in _evaluate_system_output
    metrics_scores = evaluate_system_output(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/easse/cli.py", line 184, in evaluate_system_output
    low

In [ ]:
len(read_lines(pred_path))
# pred_path


2000

In [ ]:
len(read_lines(asset_dir))

359

# Try wikilarge_mined


In [3]:
baseline_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/muss_wikilarge_mined/00/'

In [8]:
simp_asset_test_baseline = simplify_sentences(read_lines(asset_test_dir))

with open(baseline_path+'asset_test_baseline','w') as f:
  for sen in simp_asset_test_baseline:
    f.write("{}\n".format(sen))

/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size


In [9]:
simp_asset_test_baseline = simplify_sentences(read_lines(asset_valid_dir))

with open(baseline_path+'asset_valid_baseline','w') as f:
  for sen in simp_asset_test_baseline:
    f.write("{}\n".format(sen))

In [ ]:
baseline_test = baseline_path + 'asset_test_baseline'
baseline_valid = baseline_path + 'asset_valid_baseline'


ast.literal_eval(os.popen("easse evaluate -t asset_test -m 'bleu,sari,fkgl' -q < %s" %baseline_test).read())

{'bleu': 55.998,
 'sari': 43.547,
 'fkgl': 5.46,
 'quality_estimation': {'Compression ratio': 0.898,
  'Sentence splits': 1.451,
  'Levenshtein similarity': 0.7,
  'Exact copies': 0.0,
  'Additions proportion': 0.311,
  'Deletions proportion': 0.334,
  'Lexical complexity score': 7.762}}

In [ ]:
ast.literal_eval(os.popen("easse evaluate -t asset_valid -m 'bleu,sari,fkgl' -q < %s" %baseline_valid).read())

{'bleu': 55.733,
 'sari': 44.287,
 'fkgl': 5.17,
 'quality_estimation': {'Compression ratio': 0.899,
  'Sentence splits': 1.431,
  'Levenshtein similarity': 0.694,
  'Exact copies': 0.001,
  'Additions proportion': 0.325,
  'Deletions proportion': 0.348,
  'Lexical complexity score': 7.778}}

In [ ]:
# for i,j in enumerate(read_lines(baseline_valid)):
#   print(j)
#   print(read_lines(asset_valid_dir)[i])
#   print('-')

In [ ]:
read_lines(baseline_valid)[:100]

NameError: ignored

#NE_recovery


In [ ]:
pred_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/'+'model_0_local_1661643900834'+'/'+'09'+'/'+'valid.diff.complex.pred.3'

In [ ]:
ast.literal_eval(os.popen("easse evaluate -t asset_valid -m 'bleu,sari,fkgl' -q < %s" %pred_path).read())

SyntaxError: ignored

In [ ]:
len(read_lines(pred_path))

86